# First predict and select the images to be manually labeled/reviewed

In [ ]:
DATASET_DIRECTORY = '/content/drive/MyDrive/copaci'
MODELS_DIRECTORY = '/content/drive/MyDrive/copaci'
ACTUAL_MODEL_FILENAME = 'deepforest_iter2'
NEW_MODEL_FILENAME = 'deepforest_apples_2'
new_mod_file_trainer = 'deepforest_trainer'
ENV='colab'

# Install dependecies... relevant in colab

In [ ]:
if ENV == 'colab':
    !git clone https://github.com/ai-in-actiune/tree-counting-and-classification-in-images.git
    !pip install -r tree-counting-and-classification-in-images/requirements.txt
    !pip install --upgrade opencv-python setuptools==59.5.0 albumentations==1.0.3

# Train on the output from LabelImg
! Split into Train & Valid

In [ ]:
import os
workdir = '/'
if ENV == 'colab':
    from google.colab import drive
    drive.mount('/content/drive')
    workdir = '/content/tree-counting-and-classification-in-images'
elif ENV == 'local':
    workdir = '..'
os.chdir(workdir)

Mounted at /content/drive


In [ ]:
# make sure they were maually corrected before running these cells

from pathlib import Path

preds_path = Path(DATASET_DIRECTORY)
train_csv_path = preds_path / 'train' / 'labels.csv'
valid_csv_path = preds_path / 'valid' / 'labels.csv'

Prepare the csvs from the xmls

In [ ]:
from glob import glob
from tqdm import tqdm
import pandas as pd

from src.utils import xml_utils

def extract_labels_as_csvs(from_folder_path, to_file):
    path = f"{str(from_folder_path)}/*.xml"
    xmls_paths = sorted(glob(path))
    accumulator_bboxes_dfs = []
    for xml_path_str in tqdm(xmls_paths, desc="Converting xmls to csv for train eval"):
        xml_path = Path(xml_path_str)
        xml_as_df = xml_utils.xml_to_annotations(str(xml_path))
        accumulator_bboxes_dfs.append(xml_as_df)
    folder_bboxes_df = pd.concat(accumulator_bboxes_dfs)
    folder_bboxes_df.to_csv(to_file, index=False)


extract_labels_as_csvs(train_csv_path.parent, train_csv_path)
extract_labels_as_csvs(valid_csv_path.parent, valid_csv_path)

Converting xmls to csv for train eval: 100%|██████████| 50/50 [00:00<00:00, 57.24it/s]


manually create divide the manually tagged images into train/valid folders, then run the following cell

In [ ]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [ ]:
#load the modules
import os
import time
import numpy as np
from pathlib import Path

import torch
from deepforest import main 
# from deepforest import get_data
# from deepforest import utilities
# from deepforest import preprocess

In [ ]:
m = main.deepforest(num_classes=1, label_dict={'apple':0})
m.load_from_checkpoint(f'{MODELS_DIRECTORY}/{NEW_MODEL_FILENAME}/checkpoint.pl')

m.config["train"]['epochs'] = 15
m.config["batch_size"] = 4
m.config["save-snapshot"] = False
m.config["train"]["csv_file"] = str(train_csv_path)
m.config["train"]["root_dir"] = str(train_csv_path.parent)
m.config["validation"]["csv_file"] = str(valid_csv_path)
m.config["validation"]["root_dir"] = str(valid_csv_path.parent)
m.config["gpus"] = 1

Reading config file: /usr/local/lib/python3.7/dist-packages/deepforest/data/deepforest_config.yml
Reading config file: /usr/local/lib/python3.7/dist-packages/deepforest/data/deepforest_config.yml


In [ ]:
m.create_trainer()
m.trainer.fit(m)
m.evaluate(csv_file=m.config["validation"]["csv_file"], root_dir=m.config["validation"]["root_dir"])

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/configuration_validator.py:308: LightningDeprecationWarning: The `LightningModule.on_epoch_end` hook was deprecated in v1.6 and will be removed in v1.8. Please use `LightningModule.on_<train/validation/test>_epoch_end` instead.
  f"The `LightningModule.{hook}` hook was deprecated in v1.6 and"
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type      | Params
------------------------------------
0 | model | RetinaNet | 32.1 M
------------------------------------
31.9 M    Trainable params
222 K     Non-trainable params
32.1 M    Total params
128.592   Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

No predictions made


/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3441: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Training: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

{'box_precision': 0.6874928901518264,
 'box_recall': 0.8545110851900626,
 'class_recall':    label  recall  precision  size
 0      0     1.0        1.0   911,
 'predictions':              xmin        ymin         xmax        ymax  label     score  \
 0      489.531403  601.258240   573.473999  689.651062      0  0.682725   
 1     1425.833252  491.187561  1512.535278  581.436401      0  0.675515   
 2     1317.865479  124.792976  1411.205200  222.993835      0  0.670717   
 3      521.201721  231.884338   597.007996  309.979156      0  0.647505   
 4      439.928223  905.945557   527.615356  996.575623      0  0.645229   
 ...           ...         ...          ...         ...    ...       ...   
 1141   296.351135  451.122406   323.759308  479.707092      0  0.255313   
 1142   545.293579  313.897644   579.197876  348.406219      0  0.217985   
 1143   167.401627    0.000000   197.591599   26.371775      0  0.213002   
 1144   220.256546    0.893584   247.812683   29.033316      0  0

In [ ]:
save_path = Path(
    f'{MODELS_DIRECTORY}/{NEW_MODEL_FILENAME}'
)
# torch.save(m.model, save_path)
# torch.save(m.model.state_dict(), save_path)
# m.trainer.save(m.model.state_dict(), save_path)
m.trainer.save_checkpoint(save_path / "checkpoint-15.pl")
torch.save(m.model.state_dict(),
           save_path / 'model.pth')

In [ ]:
d = main.deepforest.load_from_checkpoint('/content/drive/MyDrive/copaci/deepforest_apples_2/checkpoint-15.pl')

Reading config file: /usr/local/lib/python3.7/dist-packages/deepforest/data/deepforest_config.yml


In [ ]:
im = cv2.imread('/content/drive/MyDrive/oleg/o1.jpg')

In [ ]:
d.predict_image(im)

/usr/local/lib/python3.7/dist-packages/deepforest/predict.py:34: UserWarning: Image type is uint8, transforming to float32. This assumes that the range of pixel values is 0-255, as opposed to 0-1.To suppress this warning, transform image (image.astype('float32')
  warnings.warn(f"Image type is {image.dtype}, transforming to float32. This assumes that the range of pixel values is 0-255, as opposed to 0-1.To suppress this warning, transform image (image.astype('float32')")


,xmin,ymin,xmax,ymax,label,score
0,4811.0,1482.0,5021.0,1701.0,apple,0.654635
1,4559.0,1615.0,4789.0,1857.0,apple,0.592830
2,2804.0,1264.0,2997.0,1465.0,apple,0.562095
3,2657.0,2015.0,2857.0,2224.0,apple,0.488617
4,4282.0,1080.0,4501.0,1310.0,apple,0.480064
5,4748.0,1713.0,5053.0,2019.0,apple,0.285735
6,4815.0,2592.0,5026.0,2811.0,apple,0.282151
7,912.0,1707.0,1100.0,1897.0,apple,0.233675
8,3963.0,1416.0,4160.0,1622.0,apple,0.191638
9,580.0,2749.0,784.0,2957.0,apple,0.176285


In [ ]:
m.load_from_checkpoint('/content/drive/MyDrive/copaci/deepforest_apples_2/checkpoint-15.pl')

Reading config file: /usr/local/lib/python3.7/dist-packages/deepforest/data/deepforest_config.yml


deepforest(
  (model): RetinaNet(
    (backbone): BackboneWithFPN(
      (body): IntermediateLayerGetter(
        (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        (bn1): FrozenBatchNorm2d(64, eps=0.0)
        (relu): ReLU(inplace=True)
        (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
        (layer1): Sequential(
          (0): Bottleneck(
            (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (bn1): FrozenBatchNorm2d(64, eps=0.0)
            (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (bn2): FrozenBatchNorm2d(64, eps=0.0)
            (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (bn3): FrozenBatchNorm2d(256, eps=0.0)
            (relu): ReLU(inplace=True)
            (downsample): Sequential(
              (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias

In [ ]:
import cv2
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
img = cv2.imread('/content/drive/MyDrive/oleg/o1.jpg')

In [ ]:
bimg = pd.DataFrame(m.predict_image(img))

/usr/local/lib/python3.7/dist-packages/deepforest/predict.py:34: UserWarning: Image type is uint8, transforming to float32. This assumes that the range of pixel values is 0-255, as opposed to 0-1.To suppress this warning, transform image (image.astype('float32')
  warnings.warn(f"Image type is {image.dtype}, transforming to float32. This assumes that the range of pixel values is 0-255, as opposed to 0-1.To suppress this warning, transform image (image.astype('float32')")


In [ ]:
def predict(path, model):
  im = cv2.imread(path)
  im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
  prediction = pd.DataFrame(model.predict_image(im))
  for i in range(prediction.shape[0]-1):
    x1 = int(prediction.T[i]['xmin'])
    y1 = int(prediction.T[i]['ymin'])
    x2 = int(prediction.T[i]['xmax'])
    y2 = int(prediction.T[i]['ymax'])

    cv2.rectangle(im, (x1, y1), (x2, y2), ((1-prediction.T[i]['score'])*255, prediction.T[i]['score']*255, 255), int(im.shape[0]/120/(1/(.8 + .003*im.shape[0]/100))))
  new_name = path.replace('/content/drive/MyDrive/oleg/', '_')
  cv2.imwrite(f'/content/drive/MyDrive/oleg/boxed/{new_name}.png', cv2.cvtColor(im, cv2.COLOR_RGB2BGR))
  return prediction.shape[0], im

In [ ]:
predict('/content/drive/MyDrive/oleg/o1.jpg', d)

/usr/local/lib/python3.7/dist-packages/deepforest/predict.py:34: UserWarning: Image type is uint8, transforming to float32. This assumes that the range of pixel values is 0-255, as opposed to 0-1.To suppress this warning, transform image (image.astype('float32')
  warnings.warn(f"Image type is {image.dtype}, transforming to float32. This assumes that the range of pixel values is 0-255, as opposed to 0-1.To suppress this warning, transform image (image.astype('float32')")


(15, array([[[ 42,  60,  38],
         [ 39,  57,  35],
         [ 44,  63,  41],
         ...,
         [189, 195, 185],
         [199, 201, 196],
         [200, 201, 196]],
 
        [[ 46,  61,  40],
         [ 32,  50,  28],
         [ 33,  51,  29],
         ...,
         [196, 201, 194],
         [202, 204, 199],
         [201, 202, 197]],
 
        [[ 41,  54,  36],
         [ 32,  47,  28],
         [ 39,  54,  35],
         ...,
         [210, 215, 208],
         [203, 205, 200],
         [200, 202, 197]],
 
        ...,
 
        [[ 41,  57,  46],
         [ 39,  55,  44],
         [ 35,  52,  42],
         ...,
         [ 29,  40,  34],
         [ 25,  36,  30],
         [ 25,  36,  30]],
 
        [[ 34,  50,  40],
         [ 31,  47,  37],
         [ 32,  49,  39],
         ...,
         [ 27,  38,  32],
         [ 17,  28,  22],
         [ 17,  28,  22]],
 
        [[ 28,  44,  34],
         [ 24,  40,  30],
         [ 29,  46,  36],
         ...,
         [ 25,  36,  30]

In [ ]:
p = '/content/drive/MyDrive/oleg'
g = f'{p}/*.jpg'
jpgs = glob(g)
for _ in jpgs:
  predict(_, m)

/usr/local/lib/python3.7/dist-packages/deepforest/predict.py:34: UserWarning: Image type is uint8, transforming to float32. This assumes that the range of pixel values is 0-255, as opposed to 0-1.To suppress this warning, transform image (image.astype('float32')
  warnings.warn(f"Image type is {image.dtype}, transforming to float32. This assumes that the range of pixel values is 0-255, as opposed to 0-1.To suppress this warning, transform image (image.astype('float32')")
